<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'> Modificado en 2019-1 al 2023-1 por Equipo Docente IIC2233. </font>
</p>

# Tabla de contenidos

1. [Input/Output](#Input/Output)
    1. [Bytes y encoding](#Bytes-y-encoding)
    2. [El objeto `bytes`](#El-objeto-bytes)
    3. [El objeto `bytearray`](#El-objeto-bytearray)
    4. [*Chunks*](#Chunks)
2. [I/O de archivos](#I/O-de-archivos)
3. [*Context Manager*](#Context-Manager)
4. [Cómo emular archivos para I/O](#Cómo-emular-archivos-para-I/O)

Esta semana estudiaremos detalles del uso de archivos y de la representación de *bytes* en Python. Aprenderemos que no solamente podemos almacenar *strings* como caracteres que podemos ver en la pantalla, sino que podemos también manipular la representación de más bajo nivel en el lenguaje que habla el computador: *bytes*.

# *Input/Output*

En este capítulo, estudiaremos a fondo el manejo de *bytes*, arreglos de *bytes*, archivos, y *context managers*.

## *Bytes* y *encoding*

Recordemos que los *strings* en Python son una colección de caracteres **inmutables**. 

¿Cómo se almacena un caracter en un computador? Un computador solo almacena números y los almacena en formato de Bytes. Un Byte es una secuencia de 8 *bits*, y el valor de un bit puede ser 0 ó 1. Un Byte, por lo tanto puede almacenar hasta $2^8=256$ combinaciones distintas de 0's y 1's, de manera que puede representar los números desde el 0 al 255.

Dado que un Byte solamente almacena números de 0 a 255, es válido preguntarse cómo se almacenan los caracteres. Cuando queremos representar un caracter lo que hacemos es **interpretar** el número almacenado en el Byte como un caracter. Por ejemplo podríamos decir que el Byte 0 corresponde a la letra `a`, el Byte 1 corresponde a `b`, el Byte 2 corresponde a `c`, etc, hasta cubrir todos los caracteres que queremos representar. Esa asociación se conoce como ***codificación*** o ***encoding***. Una codificación muy común es la codificación [ASCII](https://es.wikipedia.org/wiki/ASCII), que data de 1963 y asocia números (Bytes) con caracteres de la siguiente manera:

<img src="img/ascii.jpg" alt="Codificación ASCII" style="height: 400px; width:400px;"/>

La tabla ASCII muestra el valor del Byte en formato decimal (0 a 255), en formato hexadecimal (0x00 a 0xFF), y el caracter o significado correspondiente. Podemos ver que las letras mayúsculas del alfabeto corresponden desde el código 65 al código 90, las minúsculas van del código 97 al 122, y los dígitos van desde el código 48 al 57. Además los caracteres de puntuación, espacios, cambios de línea, tabulación, etc. también necesita un código.

La función `chr` de Python permite obtener el caracter correspondiente a un código decimal.


In [1]:
print(chr(105))
print(chr(105))
print(chr(99))
print(chr(50))
print(chr(50))
print(chr(51))
print(chr(51))

i
i
c
2
2
3
3


De manera equivalente, para obtener el código correspondiente a un caracter, usamos la función `ord`.

In [2]:
print(ord('B'))
print(ord('y'))
print(ord('t'))
print(ord('e'))

66
121
116
101


Para representar Bytes, si bien la tabla y la función `ord` muestran el valor decimal, es común usar dos dígitos hexadecimal. Un **dígito hexadecimal** es uno de los dígitos: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, A, B, C, D, E, F, que corresponden a un valor desde el 0 al 15 respectivamente. La representación **hexadecimal** de un Byte utiliza dos dígitos hexadecimales (valores decimales de 0 a 15) para representar los 8 bits del Byte. Para identificar que una representación es hexadecimal se suele anteponer el string `0x`. La función `hex` de Python nos permiten obtener rápidamente la representación hexadecimal de un número. Por ejemplo:

In [3]:
for i in [0, 8, 12, 15, 16, 42, 100, 255]:
    print(f"Decimal: {i}, Hexadecimal: {hex(i)}")

Decimal: 0, Hexadecimal: 0x0
Decimal: 8, Hexadecimal: 0x8
Decimal: 12, Hexadecimal: 0xc
Decimal: 15, Hexadecimal: 0xf
Decimal: 16, Hexadecimal: 0x10
Decimal: 42, Hexadecimal: 0x2a
Decimal: 100, Hexadecimal: 0x64
Decimal: 255, Hexadecimal: 0xff


Existen [muchas representaciones modernas](https://es.wikipedia.org/wiki/Codificaci%C3%B3n_de_caracteres), pero una de las más comunes y expandidas actualmente es **Unicode**, que puede representar hasta 65536 caracteres, y es el único que estaremos usando a lo largo del curso. Además la mayoría de las aplicaciones modernas utilizan por defecto esta representación.

## El objeto `bytes`

En Python los *bytes* se representan con el objeto de tipo `bytes`. Un objeto de tipo `bytes` es una secuencia **inmutable**, tal como los `str`. Para declarar que un objeto es un *byte* simplemente se pone al comienzo del objeto una `b`. Por ejemplo:

In [4]:
# Lo que está entre las comillas es un objeto de tipo bytes
# La notación \x63 indica el valor hexadecimal 63
# Este ejemplo almacena los caracteres c, l, i, c, h, é
caracteres = b"\x63\x6c\x69\x63\x68\xe9"
print(caracteres)
print(type(caracteres))

b'clich\xe9'
<class 'bytes'>


Cada secuencia `\x63` representa **1 Byte**. El Byte está descrito como un valor hexadecimal. En este caso el valor hexadecimal 63.

El símbolo escape `\x` indica que los siguientes dos caracteres después de la `x` corresponden a un *byte* usando dígitos hexadecimales. Los *bytes* que coinciden con los *bytes* de ASCII son reconocidos inmediatamente, así cuando los tratamos de imprimir aparecen correctamente (`clich`), el resto (`é`) se imprime como hexadecimal. La `b` en la impresión nos recuerda que lo que está a la derecha es un objeto de tipo `bytes`, no un `str`.

Los *bytes* a secas pueden representar cualquier entidad, desde caracteres codificados de un *string* a pixeles de una imagen. Para poder interpretar correctamente los *bytes*, necesitamos conocer la forma en que fueron codificados. Por ejemplo, un patrón binario de 8 *bits* (1 *byte*) puede corresponder a un carácter en particular si lo decodificamos usando la codificación llamada `latin1`, pero puede corresponder a un carácter completamente distinto si lo decodificamos como un carácter de tipo `utf-16`. 

In [5]:
caracteres = b"\x63\x6c\x69\x63\x68\xe9"  # Secuencia de 6 bytes
print(caracteres)

# Con bytes.decode() interpretamos los bytes utilizando la codificación latin-1 para obtener un string
print(caracteres.decode("latin-1"))

b'clich\xe9'
cliché


In [6]:
# 0x61 y 0x62 son la representación en hexadecimal de los caracteres 'a' y 'b', respectivamente
caracteres = b"\x61\x62"
print(caracteres.decode("ascii"))


ab


In [7]:
# 97 y 98 corresponden al código ASCII (en decimal) de a y b, respectivamente
caracteres = bytes((97, 98))
print(caracteres)


b'ab'


In [8]:
# Esto generará un error ya que sólo se pueden usar literales ASCII para la creación de bytes
caracteres = b"áb"


SyntaxError: bytes can only contain ASCII literal characters (2046671103.py, line 2)

El método `decode` retorna un *string* normal (Unicode). Si, por ejemplo, hubiésemos usado otro alfabeto, habríamos obtenido otro *string*.

In [9]:
caracteres = b'\x63\x6c\x69\x63\x68\xe9'
print(caracteres.decode("latin-1"))
print(caracteres.decode("iso8859-5"))
print(caracteres.decode("CP437"))
print(caracteres.decode("utf-16"))


cliché
clichщ
clichΘ
汣捩


Para codificar un *string* en un alfabeto específico, simplemente usamos el método `encode` de la clase `str`. Es necesario ingresar como argumento el conjunto de caracteres o alfabeto con que se quiere codificar.

In [10]:
caracteres = "estación"
print(caracteres.encode("UTF-8"))  # 8-bit Unicode Transformation Format
print(caracteres.encode("latin-1"))
print(caracteres.encode("CP437"))

# No se puede codificar en ASCII el caracter "ó" ya que no existe dentro
# de los 128 caracteres de ASCII
print(caracteres.encode("ascii"))


b'estaci\xc3\xb3n'
b'estaci\xf3n'
b'estaci\xa2n'


UnicodeEncodeError: 'ascii' codec can't encode character '\xf3' in position 6: ordinal not in range(128)

El método `encode` nos ofrece opciones de cómo manejar el caso en que el *string* que se quiere codificar no puede ser codificado con el alfabeto requerido. Estas opciones se ingresan a través del argumento opcional `errors`, donde los valores posibles son: `strict` (el valor por defecto), `replace`, `ignore` o `xmlcharrefreplace`. 

In [11]:
print(caracteres.encode("ascii", errors='replace'))  # en ascii se reemplaza el caracter desconocido con "?"
print(caracteres.encode("ascii", errors='ignore'))
print(caracteres.encode("ascii", errors='xmlcharrefreplace'))  # se crea una entidad xml que representa el caracter Unicode


b'estaci?n'
b'estacin'
b'estaci&#243;n'


En general, si queremos codificar un *string* y no sabemos con qué alfabeto deberíamos codificar, lo mejor es usar UTF-8, ya que los primeros 128 caracteres de UTF-8 son los mismos que en ASCII (es *backwards* compatible con ASCII). Recordar siempre que los objetos tipo *byte* son **inmutables**.

## El objeto `bytearray`

Los *bytearrays* son **arreglos (listas) de `bytes`** que, a diferencia de los `bytes` **son mutables**. Los `bytearrays` se comportan como las listas: podemos indexar con la notación de *slicing*, y también podemos ir agregando `bytes` con el método `extend`. Para construir un `bytearray` podemos ingresar un objeto `bytes` inicial.

In [12]:
ba = bytearray(b"holamundo")
print(ba)

# Podemos ocupar la notación de slicing
print(ba[3:7])
ba[4:6] = b"\x15\xa3"
print(ba)

# Podemos agregar bytes con el método extend
ba.extend(b"programa")
print(ba)


bytearray(b'holamundo')
bytearray(b'amun')
bytearray(b'hola\x15\xa3ndo')
bytearray(b'hola\x15\xa3ndoprograma')


In [13]:
# Aquí se imprime un byte, representado por un entero, el ascii que corresponde a la letra "h"
print(ba[0])
# La función bin genera un string con la representación binaria del byte
print(bin(ba[0]))
# El método zfill(r) rellena con 0's al inicio hasta completar r caracteres
print(bin(ba[0])[2:].zfill(8))


104
0b1101000
01101000


La última línea permite imprimir directamente los valores de los *bits* correspondientes al primer *byte* (representado en el literal `h` o el entero 104). El `[2:]` es para partir desde la tercera posición, ya que las primeras dos posiciones contienen los caracteres `0b`, que simplemente indica que el formato es en binario (línea anterior). Al agregar `.zfill(8)` indicamos que se usarán 8 *bits* para representar el *byte*, lo cual tiene sentido cuando hay ceros por el lado izquierdo y el *default* no los muestra (línea anterior tiene sólo 7 bits después del `0b`). 

Un caracter de un *byte* puede ser convertido a un entero usando la función `ord`.

In [14]:
# Recordemos que imprimir un elemento de un bytearray nos muestra el int que le representa
print(bytearray(b"a")[0])
# La función ord hace esta conversión directamente a partir de un byte
print(ord(b"a"))


97
97


In [15]:
# Aqui ocupamos ambas formas (int directamente y ord) para hacer cambios en un bytearray
b = bytearray(b'abcdef')
b[3] = ord(b'g')         # El caracter g tiene como código ascii el 103
b[4] = 68                # El caracter D tiene como código ascii el 68, esto sería lo mismo que ingresar b[4] = ord(b'D')
print(b)


bytearray(b'abcgDf')


### *Chunks*

Un concepto importante en el manejo de *bytes* es lo que se entiende por ***chunk***: un grupo de *bytes*. Cuando trabajamos con una cantidad importante de *bytes*, no es conveniente leerlos todos a la vez, ya que pueden ser muchos y no caber en nuestra memoria; ni tampoco uno por uno, pues requiere más operaciones de lectura de datos y puede ser muy lento. Sin embargo, podemos utilizar el punto medio: leer los *bytes* en *chunks* (grupos) de cierta cantidad de *bytes*. Por ejemplo, el siguiente *bytearray* lo leeremos en *chunks* de tamaño 4 (`TAMANO_CHUNK`):

In [16]:
muchos_bytes = bytearray(b"Una gran cantidad de bytes que quiero leer de a poquito")
TAMANO_CHUNK = 4

for i in range(0, len(muchos_bytes), TAMANO_CHUNK):
    # Aqui obtenemos nuestro chunk
    chunk = bytearray(muchos_bytes[i:i+TAMANO_CHUNK])
    print(chunk)


bytearray(b'Una ')
bytearray(b'gran')
bytearray(b' can')
bytearray(b'tida')
bytearray(b'd de')
bytearray(b' byt')
bytearray(b'es q')
bytearray(b'ue q')
bytearray(b'uier')
bytearray(b'o le')
bytearray(b'er d')
bytearray(b'e a ')
bytearray(b'poqu')
bytearray(b'ito')


## I/O de archivos

Hasta ahora, en el curso hemos operado con la lectura y escritura de archivos de texto; sin embargo, los sistemas operativos representan los archivos como secuencias de *bytes*, no como texto. Dado que leer *bytes* y convertirlos a texto es una operación muy común en archivos, los lenguajes de programación proveen maneras de manejar los *bytes* para transformarlos a una representación en *string* usando métodos de codificación (*encoding*) y decodificación (*decoding*). La función `open` nos permite, además de abrir archivos, ingresar como argumentos el set de caracteres que se usará para codificar los *bytes* y la estrategia que se debe seguir cuando aparezcan *bytes* inconsistentes con el formato:

In [17]:
# Creamos un archivo con un texto de base.
contenido = "¿Qué pasa con los tildes? ¿Y por qué no aparecen los signos de interrogación del comienzo? :("
file = open("data/archivo_ejemplo", "w", encoding='utf-8', errors="replace")
file.write(contenido)

file = open('data/archivo_ejemplo', "r", encoding='ascii', errors='replace')
print(file.read())
file.close()


��Qu�� pasa con los tildes? ��Y por qu�� no aparecen los signos de interrogaci��n del comienzo? :(


Veamos cómo cambia esto, si es que eligimos otro *encoding*.

In [18]:
file = open('data/archivo_ejemplo', "r", encoding='utf-8', errors='replace')
print(file.read())
file.close()


¿Qué pasa con los tildes? ¿Y por qué no aparecen los signos de interrogación del comienzo? :(


Vemos que ahora sí se muestran bien los tildes y los signos de interrogación, pues al leer el archivo lo estamos haciendo con el mismo *encoding* con el que fue escrito. Ahora, escribiremos en el mismo archivo, un texto distinto, y luego leeremos el archivo para ver qué ocurre.

In [19]:
contenido = "sorry pero ahora yo soy lo que habrá dentro del archivo"
file = open("data/archivo_ejemplo", "w", encoding="utf-8", errors="replace")
file.write(contenido)
file.close()
file = open('data/archivo_ejemplo', "r", encoding='utf-8', errors='replace')
print(file.read())
file.close()


sorry pero ahora yo soy lo que habrá dentro del archivo


Como puedes haber notado, el archivo se sobreescribió, y el nuevo texto en lugar de ser agregado al archivo, reemplazó lo que había. Para agregar un nuevo texto al final de un archivo ya existente, debemos cambiar el módo de apertura del archivo cambiando la `w` de *write* por una `a`, de *append*. De esta forma, al escribir en el archivo, se va a hacer al final del archivo en lugar de reemplazar el contenido anterior

In [20]:
contenido = "\nyo me agregaré al final"
file = open("data/archivo_ejemplo", "a", encoding="utf-8", errors="replace")
file.write(contenido)
file.close()

file = open('data/archivo_ejemplo', "r", encoding='utf-8', errors='replace')
print(file.read())
file.close()


sorry pero ahora yo soy lo que habrá dentro del archivo
yo me agregaré al final


Además de leer archivos de texto, podemos abrir archivos y leer sus *bytes* en lugar de texto. Para abrir un archivo como binario, simplemente debemos agregar una `b` por el lado derecho del modo de apertura. Por ejemplo, `wb` (*write bytes*) o `rb` (*read bytes*). El archivo se comportará igual que un archivo de texto, sólo que sin la codificación automática de *byte* a texto.

In [21]:
contenido = b"abcde12"
file = open("data/archivo_ejemplo_2", "wb")
file.write(contenido)
file.close()

file = open('data/archivo_ejemplo_2', "rb")
print(file.read())
file.close()


b'abcde12'


Podemos concatenar *bytes* simplemente con el operador `+`. En el siguiente ejemplo, construimos un contenido dinámico para ser escrito en un archivo de *bytes*. Después leemos una cantidad fija de *bytes* desde el mismo archivo:

In [22]:
num_lineas = 100

file = open("data/archivo_ejemplo_3", "wb")
for i in range(num_lineas):
    # A la función "bytes" debemos pasarle un iterable con el contenido a convertir
    # por eso le pasamos el entero dentro de una lista
    contenido = b"linea_" + bytes([i]) + b" abcde12 "
    file.write(contenido)
file.close()

file = open('data/archivo_ejemplo_3', "rb")
# El número dentro de la función read nos dice el número de bytes que se van a leer del archivo
print(file.read(41))
file.close()


b'linea_\x00 abcde12 linea_\x01 abcde12 linea_\x02 a'


## *Context Manager*

Dado que siempre necesitamos cerrar un archivo después de usarlo, debemos considerar la posibilidad de que ocurran excepciones mientras el archivo está abierto. Una forma clara de hacerlo es cerrar el archivo dentro de la sentencia `finally` después de un `try`; sin embargo, esto genera bastante código extra. En Python existe una forma de hacer lo mismo con menos código, a través de un *context manager*, que se encarga de ejecutar las sentencias `try` y `finally` sin la necesidad de llamarlas directamente. Sólo necesitamos llamar al archivo que queremos abrir usando la sentencia `with`. Veamos un ejemplo.

In [23]:
with open("data/archivo_ejemplo_4", "r") as file:
    contenido = file.read()


El código anterior sería equivalente a hacer lo siguiente:

In [24]:
file = open("data/archivo_ejemplo_4", "r")
try:
    contenido = file.read()
finally:
    file.close()


Si ejecutamos `dir` en un objeto de tipo archivo:

In [25]:
file = open("data/archivo_ejemplo_4", "w")
print(dir(file))
file.close()


['_CHUNK_SIZE', '__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_checkClosed', '_checkReadable', '_checkSeekable', '_checkWritable', '_finalizing', 'buffer', 'close', 'closed', 'detach', 'encoding', 'errors', 'fileno', 'flush', 'isatty', 'line_buffering', 'mode', 'name', 'newlines', 'read', 'readable', 'readline', 'readlines', 'reconfigure', 'seek', 'seekable', 'tell', 'truncate', 'writable', 'write', 'write_through', 'writelines']


Vemos que existen dos métodos llamados `__enter__` y `__exit__`. Estos dos métodos transforman el archivo en un *context manager*. El método `__exit__` asegura que el archivo será cerrado incluso si aparece una excepción mientras esté abierto. El método `__enter__` inicializa el archivo o realiza cualquier acción necesaria para ajustar el contexto del objeto.

Para asegurarnos que un archivo usará los métodos `__enter__` y `__exit__`, simplemente debemos llamar a la apertura del archivo con el método `with`.

También podemos crear nuestros propios *context managers* a partir de cualquier clase. Simplemente agregamos los métodos `__enter__` y `__exit__` y podemos llamar a nuestra clase a través del método `with`. Del siguiente ejemplo, se puede ver cómo el método `__exit__` se ejecuta una vez que nos salimos del *scope* de la sentencia `with`.

In [26]:
import string
import random


class StringUpper(list):

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        for index, char in enumerate(self):
            self[index] = char.upper()


with StringUpper() as s_upper:
    for i in range(20):
        # Aquí seleccionamos, en forma aleatoria, un ascii en minúsculas
        # y las agregamos a la lista
        s_upper.append(random.choice(string.ascii_lowercase))
    print(s_upper)

print(s_upper)


['w', 'l', 'q', 'f', 'w', 'q', 'i', 'p', 'k', 'p', 'q', 'd', 'a', 'w', 'q', 'a', 'y', 'u', 'u', 'x']
['W', 'L', 'Q', 'F', 'W', 'Q', 'I', 'P', 'K', 'P', 'Q', 'D', 'A', 'W', 'Q', 'A', 'Y', 'U', 'U', 'X']


El código anterior simplemente corresponde a una clase que hereda de la clase `list`. Al implementar los métodos `__enter__` y `__exit__`, podemos instanciar la clase a través de un *context manager*. En este ejemplo en particular, el *context manager* se encarga de transformar todos los caracteres ASCII de la lista a mayúsculas.

## Cómo emular archivos para I/O

Muchas veces tenemos que interactuar con algunos módulos de software que sólo leen y escriben sus datos desde y hacia archivos. Si queremos comunicar nuestro código que genera, por ejemplo, *strings*, para evitar tener que escribir nuestros datos en un archivo para que el otro programa los lea, podemos *emular* el tener un archivo usando los módulos de Python `StringIO` o `BytesIO`. El siguiente ejemplo muestra cómo usar estos módulos:

In [27]:
from io import StringIO, BytesIO


# aquí simulamos tener un archivo que contiene el string dado
file_in = StringIO("información como texto y más")
# aquí simulamos un archivo de Bytes para escribir la información
file_out = BytesIO()

char = file_in.read(1)
while char:
    file_out.write(char.encode("ascii", "ignore"))
    char = file_in.read(1)

buffer_ = file_out.getvalue()
print(buffer_)


b'informacin como texto y ms'


## Más información

Para conocer más sobre *bytes*, puedes visitar [este enlace](http://www.dummies.com/programming/electronics/digital-electronics-binary-basics/).